# Import Library yang dibutuhkan

In [4]:
from scipy.signal import kaiserord, lfilter, firwin, freqz
import scipy as sp
import numpy as np
import heartpy as hp
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from numpy import savetxt
from scipy.stats import entropy
from math import log, e
import os, sys
from scipy.signal import butter, iirnotch, lfilter
from scipy.stats import norm, kurtosis
from scipy.stats import skew    
import neurokit2 as nk
from scipy.signal import find_peaks
import collections

# Interval  RR + Time  Domain Features + HRV Features

In [5]:
def find_signal_peaks(arraynya, minimum=0, maximum=None, freq=500):
    dist = freq/2
    r_peaks = find_peaks(arraynya,distance=dist,prominence=(minimum,maximum))
    return r_peaks[0].tolist()

def get_rr(r_peaks, to_sec=False,sample_rate=125):
    rr_list = []
    start_stop = []
    for i in range(len(r_peaks)-2):
        rr_list.append(r_peaks[i+1]-r_peaks[i])
        start_stop.append([r_peaks[i],r_peaks[i+1]])
    if (to_sec):
        rr_list = np.divide(rr_list,sample_rate)
    return rr_list, start_stop

def shannon_entropy(data):
    bases = collections.Counter([tmp_base for tmp_base in data])
    # define distribution
    dist = [x/sum(bases.values()) for x in bases.values()]
 
    # use scipy to calculate entropy
    entropy_value = entropy(dist, base=2)
 
    return entropy_value



# Baca Dataset, Denoising, serta ekstraksi ciri

In [6]:
J=0 # jumlah file
directory_path = 'dataset/sehat'
for iter in range(0,2):
    for x in os.listdir(directory_path):
        if not x.lower().endswith('.csv'):
            continue
        J=J+1
    directory_path = 'dataset/pasien'
n = J #jumlah file
m = 17
FEAT = [] #bakal jadi Feature.csv
for i in range(n): 
    FEAT.append([0] * m) #mengisi dengan angka 0 semua 
directory_path = 'dataset/sehat'
J=-1
K=0
metoda=3
for iter in range(0,2):
    for x in os.listdir(directory_path):
        if not x.lower().endswith('.csv'):
            continue
        full_file_path = directory_path  +   '/'   + x
        J=J+1
        print ('Using file', full_file_path)
        try:
            dataraw = pd.read_csv(full_file_path,index_col='Timestamp', parse_dates=['Timestamp'])
            dataset = pd.DataFrame(dataraw['Value']) #ambil kolom value dari setiap file
        except:
            dataraw = pd.read_csv(full_file_path,index_col='timestamp', parse_dates=['timestamp'])
            dataset = pd.DataFrame(dataraw['values'])
        x1=np.array(dataset)  #ubah jadi array, namanya x1
        Dat=[]
        Dat=[0 for i in range(x1.size)] #bikin array kosong isinya 0 semua sepajang array x1
        for i in range(0,x1.size-1):
            Dat[i]=max(x1[i]) #why pakai max?
        
        
        # FIR Filter
        enhanced = hp.enhance_peaks(Dat, iterations=2)
        powerline=60
        fs = 110
        cutoff_low = 3
        nyq_rate = fs / 2.0
        width = 5.0/nyq_rate
        N, beta = kaiserord(powerline, width)
        cutoff_hz = cutoff_low
        taps = firwin(N, cutoff_hz/nyq_rate, window=('kaiser', beta))
        y_filtered = lfilter(taps, 1.0, enhanced)
            
        wd, m = hp.process(y_filtered, sample_rate=120)
        
        
        # FEATURE EXTRACTION
        try:
            #interval RR
            arr=[]
            for measure in m.keys():        
                arr.append(m[measure])
                
            #HRV Features
            #shannon entropy    
            ESH=shannon_entropy(y_filtered)
            FEAT[J][0] = ESH
            print('atas',FEAT[J][0])
            # MAD
            series = pd.Series(y_filtered)
            MAD = series.mad()
            # Kurtosis
            KURT = kurtosis(y_filtered)
            # Skewness
            SKEW = skew(y_filtered)
            print('atas',SKEW)
        
            #vlf,lf,hf
            info = nk.ppg_findpeaks(y_filtered)
            peak = info["PPG_Peaks"]
            hrv_freq = nk.hrv_frequency(peak, sampling_rate=39, normalize=True)
            VLF=hrv_freq['HRV_VLF'].values[0]
            LF=hrv_freq['HRV_LF'].values[0]
            HF=hrv_freq['HRV_HF'].values[0]
            
            #time domain features
            peaks = find_signal_peaks(y_filtered,minimum=0.2)
            rr, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=500)
            mean_ppi = np.mean(rr)
            std_ppi = np.std(rr)
            mean_signal = np.mean(y_filtered)
            std_signal = np.std(y_filtered)
            
            FEAT[J][0] = arr[0]
            FEAT[J][1] = arr[1]
            FEAT[J][2] = arr[2]
            FEAT[J][3] = arr[7]
            FEAT[J][4] = arr[12]
            FEAT[J][5] = mean_ppi
            FEAT[J][6] = std_ppi
            FEAT[J][7] = mean_signal
            FEAT[J][8] = std_signal
            FEAT[J][9] = ESH
            FEAT[J][10] = MAD
            FEAT[J][11] = KURT
            FEAT[J][12] = SKEW
            FEAT[J][13] = VLF
            FEAT[J][14] = LF
            FEAT[J][15] = HF
            FEAT[J][16] = K
        except:
            J=J-1
        
    directory_path = 'dataset/pasien'
    K=1

Using file dataset/sehat/sehat1-nizar.csv
atas 12.679919878518422
atas 1.699077420506783
Using file dataset/sehat/sehat10-bagas.csv
atas 12.66755510704113
atas 0.7716151417651285
Using file dataset/sehat/sehat11-fian.csv
atas 12.674412981837069
atas 2.1634105160453068
Using file dataset/sehat/sehat12-faizalaryo.csv
atas 12.6686634231753
atas 1.9019884102934947
Using file dataset/sehat/sehat13-dani.csv
atas 12.673750739438056
atas 1.6857134250429995
Using file dataset/sehat/sehat14 -pradana.csv
atas 12.683871868622601
atas 1.4763230190869658
Using file dataset/sehat/sehat15-rifkun.csv


C:\Users\helmi.ruslan\Anaconda3\lib\site-packages\numpy\ma\core.py:5244: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
C:\Users\helmi.ruslan\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)


atas 12.681238411777807
atas 1.6326377724653018
Using file dataset/sehat/sehat16-Nirel.csv
atas 12.759264045188791
atas -0.3467005731001139
Using file dataset/sehat/sehat17-Sesil.csv
atas 12.963437998346288
atas -0.08881717359594836
Using file dataset/sehat/sehat18-Almer.csv
atas 12.963437998346288
atas -0.08881717359594836
Using file dataset/sehat/sehat19-Aqil.csv
atas 12.89500718036866
atas 1.4513565719104917
Using file dataset/sehat/sehat2-bintang.csv
atas 12.694575537525983
atas 0.7819848769141858
Using file dataset/sehat/sehat20-thomi.csv
atas 13.192755142640372
atas 1.0957014401509415
Using file dataset/sehat/sehat21-fuad.csv
atas 12.912140856295725
atas 1.5120257484203106
Using file dataset/sehat/sehat22 - mahavira.csv
atas 12.863992637170803
atas 0.5736350309725092
Using file dataset/sehat/sehat23 - tati.csv
atas 12.75405236752893
atas 0.7438635803980507
Using file dataset/sehat/sehat24-Alvyn.csv
atas 12.96470185742045
atas -0.49898857046277706
Using file dataset/sehat/sehat25-

# Export Hasil ektraksi

In [10]:
import csv
#bikin csv
with open("Features(sk3).csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(FEAT)


# Baca Daset Hasil Ekstraksi

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import sklearn

dataset = pd.read_csv('Features(sk3).csv', names=['bpm','ibi','sdnn', 'hr_mad','breathingrate','Mean PPI', 'Std PPI', 'Mean Signal','Std signal','ESH', 'MAD', 'KURT','SKEW','VLF','LF','HF','label'])

# data_uji = pd.read_csv('Featuree2.csv', names=['bpm', 'ibi', 'sdnn','sdsd','rmssd','pnn20','pnn50','hr_mad','sd1','sd2','s','sd1/sd2','breathingrate'])
label = dataset['label']
dataset = dataset.drop(columns='label')

In [12]:
dataset.isnull().sum()


bpm              0
ibi              0
sdnn             0
hr_mad           0
breathingrate    4
Mean PPI         0
Std PPI          0
Mean Signal      0
Std signal       0
ESH              0
MAD              0
KURT             0
SKEW             0
VLF              2
LF               0
HF               0
dtype: int64

In [13]:
dataset['breathingrate']=dataset['breathingrate'].fillna(dataset['breathingrate'].mean())
dataset['VLF']=dataset['VLF'].fillna(0)


# Tahapan Klasifikasi

In [110]:
# MACHINE LEARNING
# SPLIT DATA 80% TRAIN, 20% DATA TEST
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from numpy import random

x = random.randint(100)
print(x)
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.30,random_state=39)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


print('DATA TRAIN')
print(X_train.shape)
print('LABEL TRAIN')
print(y_train.shape)
print('DATA TEST')
print(X_test.shape)
print('LABEL TEST')
print(y_test.shape)


# Import the classifier from sklearn
from sklearn.tree import DecisionTreeClassifier

# TODO: Define the classifier, and fit it to the data
model = DecisionTreeClassifier(random_state=20)
model.fit(X_train, y_train) 
y_test_pred = model.predict(X_test)

print('HASIL PREDIKSI')
print(y_test_pred)
y_train_pred = model.predict(X_train)

from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
cm2 = confusion_matrix(y_test, y_test_pred)

print('The training accuracy is', train_accuracy)
print('The test accuracy is', test_accuracy)




0
DATA TRAIN
(40, 16)
LABEL TRAIN
(40,)
DATA TEST
(18, 16)
LABEL TEST
(18,)
HASIL PREDIKSI
[0 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1]
The training accuracy is 1.0
The test accuracy is 0.9444444444444444
HASIL PREDIKSI
[0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 0 1 1]
Akurasi SVM 72.22222222222221  %


C:\Users\helmi.ruslan\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


# Hasil Sensitifiti, Spesifisiti

In [111]:
TP = cm2[1][1]

TN = cm2[0][0]
FP = cm2[0][1]
FN = cm2[1][0]
# calculate accuracy
conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))

# calculate mis-classification
conf_misclassification = 1- conf_accuracy

# calculate the sensitivity
conf_sensitivity = (TP / float(TP + FN))
# calculate the specificity
conf_specificity = (TN / float(TN + FP))

# calculate precision
conf_precision = (TN / float(TN + FP))
print(conf_specificity)
print(conf_sensitivity)

0.9
1.0
